In [1]:
# Magic
%matplotlib inline

# Libraries in use
import os
import sys
import math
import pylab as pl
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn import linear_model, neural_network, metrics
from matplotlib import collections  as mc
import string

from myutil import *
import pickle

In [2]:
# Read the results
df = pd.read_csv("out1.csv")

In [3]:
with open('asset_cov.pckl') as f:
    asset_cov = pickle.load(f)

In [4]:
df.head()

,TIMESTAMP,ID,y_pred
0,T1122,S0001,0.000000
1,T1122,S0002,0.000000
2,T1122,S0003,0.000586
3,T1122,S0004,0.000000
4,T1122,S0005,0.001723


In [5]:
timestamps = sorted(list(set(df['TIMESTAMP'])))

In [6]:
y_pred_port = []

In [7]:
import cvxopt as opt
from cvxopt import blas, solvers

# Turn off progress printing 
solvers.options['show_progress'] = False

In [8]:
for time in timestamps:
    df_current = df[(df['TIMESTAMP'] == time)]
    ids = list(df_current['ID'])
    asset_id_nums = [int(i[1:]) for i in ids]
    returns = np.array(df_current['y_pred'])
    covv = asset_cov[asset_id_nums, :]
    covv = covv[:, asset_id_nums]
    
    R = opt.matrix(returns)
    EPS = opt.matrix(covv)
    n = len(returns)
    G = opt.matrix(np.concatenate((np.eye(n), -np.eye(n)), axis=0))   # negative n x n identity matrix
    h = opt.matrix(np.concatenate((3*np.ones((n, 1)), 3*np.ones((n, 1))), axis=0)) 
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    portfolio = solvers.qp(EPS, -R, G, h, A, b)['x']
    #portfolio = solvers.qp(EPS, -R, G, h)['x']
    #portfolio = solvers.qp(EPS, -R)['x']
    
    y_pred_port.append(list(np.asarray(portfolio)))
    print time

T1122
T1123
T1124
T1125
T1126
T1127
T1128
T1129
T1130
T1131
T1132
T1133
T1134
T1135
T1136
T1137
T1138
T1139
T1140
T1141
T1142
T1143
T1144
T1145
T1146
T1147
T1148
T1149
T1150
T1151
T1152
T1153
T1154
T1155
T1156
T1157
T1158
T1159
T1160
T1161
T1162
T1163
T1164
T1165
T1166
T1167
T1168
T1169
T1170
T1171
T1172
T1173
T1174
T1175
T1176
T1177
T1178
T1179
T1180
T1181
T1182
T1183
T1184
T1185
T1186
T1187
T1188
T1189
T1190
T1191
T1192
T1193
T1194
T1195
T1196
T1197
T1198
T1199
T1200
T1201
T1202
T1203
T1204
T1205
T1206
T1207
T1208
T1209
T1210
T1211
T1212
T1213
T1214
T1215
T1216
T1217
T1218
T1219
T1220
T1221
T1222
T1223
T1224
T1225
T1226
T1227
T1228
T1229
T1230
T1231
T1232
T1233
T1234
T1235
T1236
T1237
T1238
T1239
T1240
T1241
T1242
T1243
T1244
T1245
T1246
T1247
T1248
T1249
T1250
T1251
T1252
T1253
T1254
T1255
T1256
T1257
T1258
T1259
T1260
T1261
T1262
T1263
T1264
T1265
T1266
T1267
T1268
T1269
T1270
T1271
T1272
T1273
T1274
T1275
T1276
T1277
T1278
T1279
T1280
T1281
T1282
T1283
T1284
T1285
T1286
T1287
T128

In [9]:
print type(y_pred_port[0])

<type 'list'>


In [10]:
all_lists = sum(y_pred_port, [])

In [11]:
len(all_lists)

463691

In [12]:
df_result = df.loc[:, ['TIMESTAMP', 'ID']].copy(deep=True)

In [13]:
df_result.loc[:, 'y_pred'] = all_lists

In [ ]:
#m, s, turn = sharpe_turnover(df_result)
#print m, s, turn

In [ ]:
for time in timestamps:
    preds = df_result[(df_result['TIMESTAMP'] == time)]['y_pred']
    preds = preds/sum(abs(preds))
    if(sum([1 for i in preds if abs(i) < 1e-5])) > len(preds)/2:
        print 'too many zeros'

In [14]:
df_result.to_csv('out1_markowitz.csv', index=False)

In [ ]:
df_result.shape == (463691, 3)

In [ ]:
# Read the results
df = pd.read_csv("../data/test.csv.gz")
df_result = df.loc[:, ['TIMESTAMP', 'ID']].copy(deep=True)
df_result.loc[:, 'y_pred'] = all_lists